Kaggle installation 

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! mkdir dataset
! cd dataset
! kaggle competitions download -c denoising-dirty-documents
! unzip denoising-dirty-documents.zip
! unzip test.zip
! unzip train.zip
! unzip train_cleaned.zip


In [ ]:
ROOT ="/content/"

In [ ]:
!pip install opencv-python

In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import sklearn
import random
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, Input, Conv2D, MaxPool2D, Activation, Flatten, Dropout, BatchNormalization, UpSampling2D
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, TensorBoard

# Read images

In [ ]:
PATH =""+ROOT
features = []
test_features = []
# read train images
train_path = PATH+"train/"
for file in os.listdir(train_path):
    features.append(train_path+file)

test_path = PATH+"test/"
for file in os.listdir(test_path):
    test_features.append(test_path+file)
targets = []
target_path = PATH+"train_cleaned/"
for file in os.listdir(target_path):
    targets.append(target_path+file)

# Check a few images from train, test and label sets

In [ ]:
def displayImage(image_name=None, img_title=""):
    if image_name:
        from matplotlib import image
        img = image.imread(image_name) # train image 
        plt.imshow(img)
        plt.title(img_title)
        plt.gray()
        plt.show()
    
    return None


# check with random images from train, test and target sets

In [ ]:
displayImage(features[0],"Image with Noise") # Train Image

In [ ]:
displayImage(targets[0],"Image after denoising") # Target Image

# For feeding into the ML model, all images should have same shape. We will check if all iamges have same dimension.


In [ ]:
def checkImageDimension(images):
    height = []
    width = []
    for image in images:
        img = cv2.imread(image)
        height.append(img.shape[0])
        width.append(img.shape[1])
    
    print(f'(Min Height, Max Height):{min(height), max(height)}')
    print(f'(Min Width, Max Width):{min(width), max(width)}')

In [ ]:
checkImageDimension(features)

# It's clearlly visible that some images have different heights. So, it would be wise to make the images of same height. So, our target images dimension would be: 258x540x3

In [ ]:
# resize and normalize images between 0 and 1

def resizeNormalizeImages(images):
    new_shape = (540,258)
    preprocessed_images = []
    for image in images:
        img = cv2.imread(image,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, new_shape)
        img = img/255.
        img = np.expand_dims(img, axis=-1)
        preprocessed_images.append(img)
    return preprocessed_images



In [ ]:
# generate train, test and target sets
processed_features = np.array(resizeNormalizeImages(features))
processed_targets = np.array(resizeNormalizeImages(targets))
processed_test = np.array(resizeNormalizeImages(test_features))

In [ ]:
processed_features.shape

# Implement Machine Learning model:
## Split data(train/target sets) set for training and validation


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(processed_features, processed_targets,test_size=0.3, random_state=540)

In [ ]:
# check training and validation dataset sizes
print(f'Training set Size: {X_train.shape}')
print(f'Validation set Size: {X_val.shape}')

# Generate Models: Encoder Decoder

In [ ]:
# Input Layer 
input_layer = Input(shape=(258,540,1))
model = Sequential()

# Encoder Part:
model.add(Conv2D(input_shape=(258,540,1), filters=256,kernel_size=(3,3), activation='relu', padding='same', name="en_conv_1"))
model.add(BatchNormalization(name="en_bn_1"))
model.add(MaxPool2D((2,2), padding='same',name='en_max_pool_1'))
#decoder
model.add(Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding='same', name='de_conv_1'))
model.add(UpSampling2D((2,2), name='upsampling_1'))
model.add(Conv2D(filters=1, kernel_size=(3,3), activation='sigmoid', padding='same', name='de_conv_2'))

model.summary()


In [ ]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['RootMeanSquaredError'])
history = model.fit(X_train,y_train, batch_size=16, epochs=200, validation_data=(X_val,y_val))

In [ ]:
# plot model loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training/Validation Loss')
plt.legend(['Training Loss','Validation Loss'])
plt.show()

In [ ]:
predictions = model.predict(processed_test)
predictions.shape

In [ ]:
predictions.shape[0]

In [ ]:
for i in range(0,processed_test.shape[0]):
  f, axs = plt.subplots(1,2,figsize=(10,5))
  axs[0].imshow(processed_test[i].reshape(258,540),cmap='gray')
  axs[0].title.set_text('With Noise Image '+str(i+1))
  axs[1].imshow(predictions[i].reshape(258,540),cmap='gray')
  axs[1].title.set_text("After denoising Image "+str(i+1))
